## Read the training data and import library

In [607]:
import numpy as np
import pandas as pd
import pickle

In [608]:
class standardScaler():
    def fit(self, xss):
        self.mean = np.mean(xss, axis=0)
        self.sd = np.std(xss, axis=0)

    def transform(self, xss):
        xss = (xss-self.mean)/(self.sd)
        return(xss)

In [609]:
from sklearn.preprocessing import PolynomialFeatures 
def polyTransform(xss):
#     xss = np.column_stack([xss, xss**2])
    poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
    xss = poly.fit_transform(xss)
    return(xss)

## Process the data 9 hours

In [610]:
def generate_Initial_Parameters(outliers=[],scale=True):
    # read data
    data = pd.read_csv('data/train.csv', encoding='Big5')
    data = data.replace('NR', '0')
    data = np.array(data)
    
    # feature variable
    hour = 9
    feature_num = 18
    day_per_month = 20
    per_month_row = feature_num * day_per_month
    total_month = int(len(data)/per_month_row) 
    
    month_data = []
    for i in range(total_month):
        l = data[i * per_month_row: i * per_month_row  + per_month_row]
        month_data.append(l)  

    hour_data = []
    for i in range(len(month_data)):
        tmp = []
        for j in range(len(month_data[i])):
            tmp.append(month_data[i][j])
        hour_data.append(tmp)

    total = []
    for i in range(len(hour_data)):
        df = pd.DataFrame(hour_data[i])
        row, col = df.shape
        tmp = None
        for j in range(day_per_month):
            per_day = df.iloc[j*feature_num:j*feature_num+feature_num,:]
            per_day = per_day.iloc[:,3:]

            if tmp is not None:
                tmp = pd.concat([tmp.reset_index(drop=True), per_day.reset_index(drop=True)], axis=1, ignore_index=True)
            else:
                tmp = pd.DataFrame(per_day)   

        total.append(tmp)
        
    xss = []
    yss = []
    ori_xss = []
    ori_yss = []

    for i in range(len(total)):
        df = total[i]
        row, col = df.shape
        for j in range(col-hour):
            xs = df.iloc[:,j:j+hour]
            xs = xs.values.ravel()
            ys = float(df.iloc[9,j+hour])

            ori_xss.append(xs)
            ori_yss.append(float(ys))

            if(ys < 0):
                continue
                
            xss.append(xs)
            yss.append(float(ys))

    xss = pd.DataFrame(xss).values.astype(np.float)
    yss = np.array(yss)
    ori_xss = pd.DataFrame(ori_xss).values.astype(np.float)
    ori_yss = np.array(ori_yss)

    if scale:
        ## feature scaling
        scaler = standardScaler()
        scaler.fit(xss)
        xss = scaler.transform(xss)
        row, col = xss.shape
    
    if outliers:
        xss = np.array([ xss[i] for i in range(len(xss)) if i not in outlier])
        yss = np.array([ yss[i] for i in range(len(yss)) if i not in outlier])
        
    return ({'xss': xss, 'yss': yss, 'ori_xss': ori_xss, 'ori_yss': ori_yss, 'scaler': scaler})

## Feature Selection

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    
def calculate_vif_(X, thresh=150):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            variables = np.delete(variables, maxloc)
            dropped=True

    print('Remaining variables:')
    print(X.columns[variables])
    return (X.columns[variables], X[cols[variables]])

In [448]:
param = generate_Initial_Parameters(outliers=outlier)
xss = param['xss']
yss = param['yss']
scaler = param['scaler']
print(xss.shape)

(4029, 162)


In [166]:
xss = polyTransform(xss)
print(xss.shape)

(5518, 298)


In [ ]:
res = calculate_vif_(X_train)
selected_var, xss = res[0], res[1].values

## Select only past pm2.5 as factor

In [ ]:
selected_var = np.array([  0,   2,   4,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29, 30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42, 43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77, 78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90, 91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,  156, 157, 158, 159, 160, 161]) 

In [399]:
# selected_var = np.array([(i)*feature_num +9 for i in range(9)]
X_train = pd.DataFrame(xss)
X_train = X_train[selected_var]
xss = X_train.values

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(xss, yss, test_size=0.2, random_state=20)

## Train the model. Don't forget to remove the clear_output

In [168]:
from IPython.display import clear_output

# def gradientDescent(xss, yss , alpha):
#     lr = 1
#     max_iter = 10 ** 3
#     epochs = 200
    
#     ## get bias
#     xss = np.column_stack(([1] * len(xss) ,xss))
#     num = xss.shape[1]
#     w = np.zeros(num)
#     w_lr = np.zeros(num)
    
#     for t in range(epochs):
#         w_grad = None
#         for m in range(max_iter):
#             predict = np.dot(xss,w)
#             w_grad = -(2 * np.dot(xss.T,(yss - predict))) + (alpha * np.sum(w**2) * 2)
#             w_lr = w_lr + w_grad ** 2
#             w = w - lr/np.sqrt(w_lr) * w_grad

#         clear_output()
#         print(t)
#         print(np.sqrt(np.mean([ x*x for x in (yss-predict)])))
        
#     return (w)

In [13]:
from IPython.display import clear_output

def gradientDescent(xss, yss , alpha):
    xss = np.column_stack(([1] * len(xss) ,xss))
    AtA =(alpha * np.identity(len(xss[0])) + np.dot(xss.T, xss)) 
    inv = np.linalg.inv(AtA)
    w =  np.dot(inv, xss.T)
    w = np.dot(w,yss)
    return(w)

In [297]:
# alpha_list = [10, 6, 5, 4, 3, 2, 1, 0.5, 0.3, 0.2, 0.1, 0.01, 0.001, 0.0001]
alpha_list = [0]
his = []
for alpha in alpha_list:
    w = gradientDescent(X_train, Y_train, alpha)
    xss_tmp = np.column_stack(([1] * len(X_test) ,X_test))
    yss_tmp = Y_test

    predict = np.dot(xss_tmp ,w)

    rmse = (np.sqrt(np.mean([ x*x for x in (yss_tmp-predict)])))
    his.append(rmse)
    
print(his)
print(his.index(min(his)))

[4.915493308054814]
0


In [436]:
w = gradientDescent(xss, yss, alpha = 0)

In [476]:
test = pd.read_csv('data/test.csv', encoding='Big5', header=None)
test = test.replace('NR', '0')
file = '0306-4'

In [478]:
import pickle
dic = {'w': w, 'scaler': scaler, 'selected_var' : selected_var, 'alpha' : alpha}
with open('model/'+file+'.pkl', 'wb') as f:
    pickle.dump(dic, f)

In [ ]:
# with open('model/0306-2.pkl', 'rb') as f:
#     dic = pickle.load(f)

# w = dic['w']
# scaler = dic['scaler']
# selected_var = dic['selected_var']

ans = pd.read_csv('data/sampleSubmission.csv', encoding='Big5')
test_feature = 18
total_test = []
row, col = test.shape
test_number = int(row/test_feature)

for i in range(test_number):
    df = test.iloc[i*test_feature:(i+1)*test_feature, 2:]
    xs = df.values.ravel().astype(np.float)
    xs = (scaler.transform([xs])[0])
    xs = np.array([[xs[i] for i in range(len(xs)) if i in selected_var]])
    xs = polyTransform(xs)[0]
    xs = np.concatenate(([1], xs))
    val = np.dot(xs,w)
#     val = (round(val,2))
    ans.iloc[i,1] = val
    
ans.to_csv('data/'+file+'.csv',index=False)

In [ ]:
import matplotlib.pyplot as plt
# with open('model/0304-3.pkl', 'rb') as f:
# with open('model/0305-1.pkl', 'rb') as f:
#     dic = pickle.load(f)

# w = dic['w']
# scaler = dic['scaler']
# selected_var = dic['selected_var']
his = []
his_train = []
alphas = np.logspace(-5, 3, 5)
min_rmse = 10000


param = generate_Initial_Parameters(outliers=[])
xss = param['xss']
yss = param['yss']
scaler = param['scaler']
xss = xss[:,selected_var]
xss = polyTransform(xss)
res = {'w' : [], 'alpha': 0, 'k': -1 }

for m in range(len(alphas)):
    for k in range(15,50,5):
        alpha = alphas[m]
        if k % 10 == 0:
            clear_output()

        ## take all 
        param = generate_Initial_Parameters()
        xss = param['xss']
        yss = param['yss']

        xss = xss[:,selected_var]
        xss = polyTransform(xss)
        X_train, X_test, Y_train, Y_test = train_test_split(xss, yss, test_size=0.2, random_state=33)
        w = gradientDescent(X_train, Y_train, alpha = alpha)
        
        val = np.column_stack(([1] * len(X_train) ,X_train))
        predict = np.dot(val,w)
        predict = np.array([round(i,0) for i in predict])
        Y_train = np.array(Y_train)

        error = (predict-Y_train)
        rmse = np.sqrt(np.mean(error**2))
        his_train.append(rmse)
#         print('training rmse =', rmse, end=',')
        outlier = [ i for i in range(len(error)) if abs(error[i]) >= k]

        X_train = [ X_train[i] for i in range(len(X_train)) if i not in outlier]
        Y_train = [ Y_train[i] for i in range(len(Y_train)) if i not in outlier]
        
        w = gradientDescent(X_train, Y_train, alpha = alpha)
        val = np.column_stack(([1] * len(X_test) ,X_test))
        predict = np.dot(val,w)
        predict = np.array([round(i,0) for i in predict])
        Y_test = np.array(Y_test)
        
        error = (predict-Y_test)
        rmse = np.sqrt(np.mean(error**2))
        
        min_rmse = min(rmse, min_rmse)
        if min_rmse == rmse:
            res['w'] = w
            res['alpha'] = alpha
            res['k'] = k
            
#         print('rmse =', rmse)
        his.append(rmse)

In [ ]:
plt.plot(his[:20])
plt.plot(his_train[:20])
# i = 0
# plt.plot(his[i*8:(i+1)*8])
# plt.plot(his_train[i*8:(i+1)*8])

In [551]:
list(range(10,50,5))

[10, 15, 20, 25, 30, 35, 40, 45]

In [578]:
np.column_stack([his,his_train])

array([[5.46841581, 5.66292769],
       [5.48431204, 5.66292769],
       [5.50219112, 5.66292769],
       [5.48007755, 5.66292769],
       [5.51418625, 5.66292769],
       [5.51418625, 5.66292769],
       [5.51418625, 5.66292769],
       [5.46841581, 5.66292769],
       [5.48431204, 5.66292769],
       [5.50219112, 5.66292769],
       [5.48007755, 5.66292769],
       [5.51418625, 5.66292769],
       [5.51418625, 5.66292769],
       [5.51418625, 5.66292769],
       [5.46857908, 5.66229698],
       [5.48178803, 5.66229698],
       [5.50227226, 5.66229698],
       [5.47852952, 5.66229698],
       [5.51588615, 5.66229698],
       [5.51588615, 5.66229698],
       [5.51588615, 5.66229698],
       [5.43459487, 5.66797079],
       [5.47608437, 5.66797079],
       [5.47110919, 5.66797079],
       [5.46596613, 5.66797079],
       [5.48919393, 5.66797079],
       [5.48919393, 5.66797079],
       [5.48919393, 5.66797079],
       [8.57331824, 7.88308825],
       [8.0123231 , 7.88308825],
       [7.

In [582]:
res['w']

array([ 2.14327164e+01,  9.42967919e-02,  1.77463578e-01, -3.37513246e-01,
       -1.41598001e+00,  1.48963463e+00, -5.01948146e-01,  6.22195351e-02,
        9.01952569e-02,  1.12970576e-01, -1.50122026e-01,  1.99187906e-01,
       -3.36505704e-01, -3.27081288e-02,  3.12005043e-01,  6.73955916e-02,
       -4.85585377e-02,  2.12013325e-01, -2.32487687e-01,  3.00238751e-01,
       -1.80666205e-01,  1.68637481e-02,  3.83480525e-02,  2.95507428e-01,
       -5.72547191e-01,  5.87278685e-01, -5.60493178e-01,  3.28772890e-01,
        2.46536530e-01, -3.94433444e-01, -4.24959805e-01,  1.47216014e-01,
        1.77650362e-02,  4.58895622e-02, -7.99386140e-02,  3.57781551e-01,
       -9.36817060e-02,  1.45570388e-03, -1.98774201e-01,  2.77368106e-02,
        9.51228082e-02,  1.16620483e-02,  3.49364882e-02, -2.00154645e-01,
       -2.31116927e-01, -1.81887832e-01,  1.39383100e-01, -1.89915867e-01,
       -2.94774636e-01, -1.67504589e-01,  1.49464044e+00,  1.74603863e-01,
        1.87745487e-01, -

In [580]:
k=res['k']
al=res['alpha']

param = generate_Initial_Parameters(outliers=[])
xss = param['xss']
xss = xss[:,selected_var]
yss = param['yss']
scaler = param['scaler']

val = np.column_stack([([1] * len(xss)) , xss])
predict = np.dot(val,w)
predict = np.array([round(i,0) for i in predict])
yss = np.array(yss)
error = (predict-yss)
outlier = [ i for i in range(len(error)) if abs(error[i]) >= k]
print(len(outlier))

param = generate_Initial_Parameters(outliers=outlier)
xss = param['xss']
xss = xss[:,selected_var]
yss = param['yss']
scaler = param['scaler']

val = np.column_stack(([1] * len(xss) ,xss))
predict = np.dot(val,w)
predict = np.array([round(i,0) for i in predict])
yss = np.array(yss)
w = gradientDescent(xss, yss , alpha = 10)
print(len(w))

347
150


In [561]:
w = gradientDescent(xss, yss, alpha = alpha)
        
## take all 
param = generate_Initial_Parameters()
xss = param['xss']
yss = param['yss']

xss = xss[:,selected_var]
xss = polyTransform(xss)

val = np.column_stack(([1] * len(xss) ,xss))
predict = np.dot(val,w)
predict = [round(i,0) for i in predict]

error = (predict-yss)
rmse =  np.sqrt(np.mean(error ** 2)) 
min_rmse = min(min_rmse,rmse)


if min_rmse == rmse:
    ans['w'] = w
    ans['alpha'] = alpha
    ans['outlier'] = outlier
print('rmse =', rmse)
his.append(rmse)

outlier = [ i for i in range(len(error)) if abs(error[i]) >= k]
print('error num', len(outlier))

xss = [ xss[i] for i in range(len(xss)) if i not in outlier]
yss = [ yss[i] for i in range(len(yss)) if i not in outlier]

print(len(xss))
xss = pd.DataFrame(xss)

rmse = 9.542751549886287
error num 1541
4059
